In [78]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import seaborn as sns
from pprint import pprint
from sqlalchemy import create_engine

# File to Load (Remember to Change These)
Zillow_df = pd.read_excel("Zillow Data.xlsx")
Realtor_df = pd.read_csv("RDC_Inventory_Core_Metrics_Zip_History 91910 92110.csv")



In [79]:
Zillow_df.head()

,Website,Date,Postal_Code,City,median_listing_price
0,Zillow,202009,91910,Chula Vista,629050.0
1,Zillow,202009,92110,San Diego,660050.0
2,Zillow,202008,91910,Chula Vista,629050.0
3,Zillow,202008,92110,San Diego,614050.0
4,Zillow,202007,91910,Chula Vista,620050.0


In [80]:
Realtor_df.head()

,Website,Date,Postal_Code,City,median_listing_price
0,Realtor,202010,91910,Chula Vista,619050.0
1,Realtor,202010,92110,San Diego,650050.0
2,Realtor,202009,92110,San Diego,619050.0
3,Realtor,202009,91910,Chula Vista,604050.0
4,Realtor,202008,91910,Chula Vista,610050.0


In [81]:
realtor_data_complete_df = pd.merge(Zillow_df, Realtor_df, how="outer", left_on=["Postal_Code", "Date","City"], right_on=["Postal_Code", "Date","City"])

In [82]:
realtor_data_complete_df.head()

,Website_x,Date,Postal_Code,City,median_listing_price_x,Website_y,median_listing_price_y
0,Zillow,202009,91910,Chula Vista,629050.0,Realtor,604050.0
1,Zillow,202009,92110,San Diego,660050.0,Realtor,619050.0
2,Zillow,202008,91910,Chula Vista,629050.0,Realtor,610050.0
3,Zillow,202008,92110,San Diego,614050.0,Realtor,600050.0
4,Zillow,202007,91910,Chula Vista,620050.0,Realtor,614550.0


In [83]:
realtor_data_complete_df = realtor_data_complete_df.dropna(how='any')

In [84]:
index = realtor_data_complete_df.index
number_of_rows = len(index)
print(number_of_rows)

102


In [85]:
realtor_data_complete_df.dtypes

Website_x                  object
Date                        int64
Postal_Code                 int64
City                       object
median_listing_price_x    float64
Website_y                  object
median_listing_price_y    float64
dtype: object

In [86]:
realtor_data_complete_df['Postal_Code'] = realtor_data_complete_df['Postal_Code'].astype('int64')
realtor_data_complete_df['Date'] = pd.to_datetime(realtor_data_complete_df['Date'].astype(str), format='%Y%m')


In [87]:
realtor_data_complete_df.dtypes

Website_x                         object
Date                      datetime64[ns]
Postal_Code                        int64
City                              object
median_listing_price_x           float64
Website_y                         object
median_listing_price_y           float64
dtype: object

In [88]:
realtor_data_complete_df.head()

,Website_x,Date,Postal_Code,City,median_listing_price_x,Website_y,median_listing_price_y
0,Zillow,2020-09-01,91910,Chula Vista,629050.0,Realtor,604050.0
1,Zillow,2020-09-01,92110,San Diego,660050.0,Realtor,619050.0
2,Zillow,2020-08-01,91910,Chula Vista,629050.0,Realtor,610050.0
3,Zillow,2020-08-01,92110,San Diego,614050.0,Realtor,600050.0
4,Zillow,2020-07-01,91910,Chula Vista,620050.0,Realtor,614550.0


In [89]:
clean_realtor_df= realtor_data_complete_df.rename(columns={
    'median_listing_price_x': 'zillow_median_listing_price', 'median_listing_price_y': 'realtor_median_listing_price', 'Website_x': 'website_x', 'Website_y': 'website_y'})


In [90]:
clean_realtor_df.head()

,website_x,Date,Postal_Code,City,zillow_median_listing_price,website_y,realtor_median_listing_price
0,Zillow,2020-09-01,91910,Chula Vista,629050.0,Realtor,604050.0
1,Zillow,2020-09-01,92110,San Diego,660050.0,Realtor,619050.0
2,Zillow,2020-08-01,91910,Chula Vista,629050.0,Realtor,610050.0
3,Zillow,2020-08-01,92110,San Diego,614050.0,Realtor,600050.0
4,Zillow,2020-07-01,91910,Chula Vista,620050.0,Realtor,614550.0


In [93]:
clean_realtor_df = clean_realtor_df[['Date', 'Postal_Code', 'City', 'website_x', 'zillow_median_listing_price', 'website_y', 'realtor_median_listing_price']]

In [94]:
clean_realtor_df.head()

,Date,Postal_Code,City,website_x,zillow_median_listing_price,website_y,realtor_median_listing_price
0,2020-09-01,91910,Chula Vista,Zillow,629050.0,Realtor,604050.0
1,2020-09-01,92110,San Diego,Zillow,660050.0,Realtor,619050.0
2,2020-08-01,91910,Chula Vista,Zillow,629050.0,Realtor,610050.0
3,2020-08-01,92110,San Diego,Zillow,614050.0,Realtor,600050.0
4,2020-07-01,91910,Chula Vista,Zillow,620050.0,Realtor,614550.0


In [98]:
zillow_info_df = clean_realtor_df[['website_x', 'zillow_median_listing_price']].copy()
zillow_info_df.head()

,website_x,zillow_median_listing_price
0,Zillow,629050.0
1,Zillow,660050.0
2,Zillow,629050.0
3,Zillow,614050.0
4,Zillow,620050.0


In [99]:
realtor_info_df = clean_realtor_df[['website_y', 'realtor_median_listing_price']].copy()
realtor_info_df.head()

,website_y,realtor_median_listing_price
0,Realtor,604050.0
1,Realtor,619050.0
2,Realtor,610050.0
3,Realtor,600050.0
4,Realtor,614550.0


In [100]:
house_location_df = clean_realtor_df[['Postal_Code', 'City']].copy()
house_location_df.head()

,Postal_Code,City
0,91910,Chula Vista
1,92110,San Diego
2,91910,Chula Vista
3,92110,San Diego
4,91910,Chula Vista


In [101]:
date_df = clean_realtor_df[['Date']].copy()
date_df.head()

,Date
0,2020-09-01
1,2020-09-01
2,2020-08-01
3,2020-08-01
4,2020-07-01


In [103]:
rds_connection_string = "postgres:BUChoy123!!!@localhost:5432/realtor_house_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [109]:
engine.table_names()

['zillow_info', 'realtor_info', 'house_location', 'date']

In [105]:
zillow_info_df.to_sql(name='zillow_info', con=engine, if_exists='append', index=False)

In [106]:
realtor_info_df.to_sql(name='realtor_info', con=engine, if_exists='append', index=False)

In [107]:
house_location_df.to_sql(name='house_location', con=engine, if_exists='append', index=False)

In [108]:
date_df.to_sql(name='date', con=engine, if_exists='append', index=False)